In [53]:
import networkx as nx
from manim import *
import math

config.media_width = "100%"
config.verbosity = "WARNING"

In [54]:
# Bitwise operations needed
def flip_bit(a, n):
	# a: number to modify
	# n: bit to flip (starting at 0)
	x = 1<<n
	return a^x

In [55]:
# Test bitwise operations
assert(flip_bit(7, 0) == 6)
assert(flip_bit(7, 1) == 5)
assert(flip_bit(7, 2) == 3)
assert(flip_bit(7, 3) == 15)

In [244]:
def generic_run(selfi, network, **kwargs):
    null_label = kwargs['null_label'] if 'null_label' in kwargs else '-'
    delay = kwargs['delay'] if 'delay' in kwargs else 1
    (Network, layout) = network.get()

    unet = Network.to_undirected()
    network.init_computation()
    finished = False
    time = 0
    while not finished:
        print(f'Iteration {time}')
        if time == 0:
            G = Graph.from_networkx(Network, layout=layout, labels={i : Network.nodes[i]['weight']['value'] for i in Network}, 
                                    edge_config={(u,v): {'stroke_color': (WHITE if d['weight']['carry'] != null_label else GREEN),
                                                         'stroke_width': (6 if d['weight']['carry'] != null_label else 1)} 
                                                         for u, v, d in Network.edges(data = True)},
                                    vertex_config={u: {'fill_color': (WHITE if Network.nodes[u]['weight']['value'] != null_label else BLACK),
                                                       'stroke_width': 2, 'stroke_color': WHITE}
                                                   for u in Network.nodes})
            selfi.add(G)
            counter = Text(str(time)).to_edge(UL)
            selfi.add(counter)
            try:
                list = network.additional_items_to_draw()
                for item in list:
                    if (item[0] == "RectangleAroundNodes"):
                        group = Group(*[G[x] for x in item[1]])
                        selfi.add(SurroundingRectangle(group))
            except Exception as e:
                print("Nothing to add:", e)
        else:
            selfi.wait(delay)
            G1 = Graph.from_networkx(Network, layout=layout, labels={i : Network.nodes[i]['weight']['value'] for i in Network}, 
                                    edge_config={(u,v): {'stroke_color': (WHITE if d['weight']['carry'] != null_label else GREEN),
                                                         'stroke_width': (6 if d['weight']['carry'] != null_label else 1)} 
                                                         for u, v, d in Network.edges(data = True)},
                                    vertex_config={u: {'fill_color': (WHITE if Network.nodes[u]['weight']['value'] != null_label else BLACK),
                                                       'stroke_width': 2, 'stroke_color': WHITE}
                                                   for u in Network.nodes})

            new_texts_from = {(u, v): Text(str(Network[u][v]['weight']['carry']), color=RED, font_size=32)
                        for u, v, d in Network.edges(data=True) if d['weight']['carry'] != null_label}
            new_texts_to = {(u, v): Text(str(Network[u][v]['weight']['carry']), color=RED, font_size=32)
                        for u, v, d in Network.edges(data=True) if d['weight']['carry'] != null_label}
            selfi.play(Transform(G, G1), *[Transform(new_texts_from[(u, v)].move_to(G[u]), new_texts_to[(u, v)].move_to(G[v])) 
                                           for u, v, d in Network.edges(data=True) if d['weight']['carry'] != null_label], 
                        path_arc = 70 * DEGREES, run_time=1.5*delay)
            selfi.play(Transform(counter, Text(str(time)).to_edge(UL)) ,*[FadeOut(new_texts_from[(u, v)]) for u, v, d in Network.edges(data=True) if d['weight']['carry'] != null_label])
        finished = not network.compute_iteration()
        time = time+1
    selfi.wait(delay)


## Odd-Even Sort

In [245]:
class odd_even:
    def node_id(self, line, stage, comparator):
        assert(comparator>=0 and comparator < 4)
        assert(line < self.lines)
        assert(stage < self.lines)
        return (line<<int(math.log2(self.lines)*2+2)) + (stage<<int(math.log2(self.lines)+2)) + comparator
    
    def items_from_node_id(self, nodeid):
        return ((nodeid &  ((1<<int(math.log2(self.lines)))-1)<<int(math.log2(self.lines)*2+2)) >> int(math.log2(self.lines)*2+2),
                (nodeid &  ((1<<int(math.log2(self.lines)))-1)<<int(math.log2(self.lines)+2)) >> int(math.log2(self.lines)+2),
                nodeid&3)

    def input_node(self, line):
        return -(line+1)
    
    def inputs(self):
        return range(-1, -(self.lines+1), -1)
    
    def output_node(self, line):
        return -(line+1)-2*self.lines

    def outputs(self):
        return range(-1-2*self.lines, -(self.lines+1)-2*self.lines, -1)
    
    def __init__(self, nlines, **kwargs):
        #print(kwargs)
        self.null_label = kwargs['null_label'] if 'null_label' in kwargs else '-'
        self.verbose = kwargs['verbose'] if 'verbose' in kwargs else False

        # nbits is the number of bits of the operation
        # N is the number of nodes in the tree (2^n-1). Nodes in the tree will be from 0 to 2^n-2
        self.lines = nlines
        self.N = 2*nlines-1

        self.OddEven = nx.DiGraph()
        self.layout = {}
        for stage in range(0, self.lines):
            for line in range((stage%2), self.lines-(stage%2), 2):
                #print(f'{stage}, {line}: {self.node_id(line, stage, 0)} {self.node_id(line, stage, 1)} {self.node_id(line+1, stage, 0)} {self.node_id(line+1, stage, 1)}')
                if stage%2 == 0:
                    self.OddEven.add_node(self.node_id(line, stage, 0), weight = {'value': self.null_label})
                    self.OddEven.add_node(self.node_id(line, stage, 1), weight = {'value': self.null_label})
                    self.OddEven.add_node(self.node_id(line+1, stage, 0), weight = {'value': self.null_label})
                    self.OddEven.add_node(self.node_id(line+1, stage, 1), weight = {'value': self.null_label})
                    self.OddEven.add_edge(self.node_id(line, stage, 0), self.node_id(line, stage, 1), weight={"carry": self.null_label, "epoch": 0})
                    self.OddEven.add_edge(self.node_id(line+1, stage, 0), self.node_id(line+1, stage, 1), weight={"carry": self.null_label, "epoch": 0})
                    self.OddEven.add_edge(self.node_id(line, stage, 0), self.node_id(line+1, stage, 1), weight={"carry": self.null_label, "epoch": 0})
                    self.OddEven.add_edge(self.node_id(line+1, stage, 0), self.node_id(line, stage, 1), weight={"carry": self.null_label, "epoch": 0})
                    self.layout[self.node_id(line, stage, 0)] = [2*stage - self.lines+1, -(line - self.lines/2+1), 0]
                    self.layout[self.node_id(line, stage, 1)] = [2*stage+1 - self.lines+1, -(line - self.lines/2+1), 0]
                    self.layout[self.node_id(line+1, stage, 0)] = [2*stage - self.lines+1, -(line+1 - self.lines/2+1), 0]
                    self.layout[self.node_id(line+1, stage, 1)] = [2*stage+1 - self.lines+1, -(line+1 - self.lines/2+1), 0]
                else:
                    self.OddEven.add_node(self.node_id(line, stage, 0), weight = {'value': self.null_label})
                    self.OddEven.add_node(self.node_id(line, stage, 1), weight = {'value': self.null_label})
                    self.OddEven.add_node(self.node_id(line+1, stage, 0), weight = {'value': self.null_label})
                    self.OddEven.add_node(self.node_id(line+1, stage, 1), weight = {'value': self.null_label})
                    self.OddEven.add_edge(self.node_id(line, stage, 0), self.node_id(line, stage, 1), weight={"carry": self.null_label, "epoch": 0})
                    self.OddEven.add_edge(self.node_id(line+1, stage, 0), self.node_id(line+1, stage, 1), weight={"carry": self.null_label, "epoch": 0})
                    self.OddEven.add_edge(self.node_id(line, stage, 0), self.node_id(line+1, stage, 1), weight={"carry": self.null_label, "epoch": 0})
                    self.OddEven.add_edge(self.node_id(line+1, stage, 0), self.node_id(line, stage, 1), weight={"carry": self.null_label, "epoch": 0})
                    self.layout[self.node_id(line, stage, 0)] = [2*stage - self.lines+1, -(line - self.lines/2+1), 0]
                    self.layout[self.node_id(line, stage, 1)] = [2*stage+1 - self.lines+1, -(line - self.lines/2+1), 0]
                    self.layout[self.node_id(line+1, stage, 0)] = [2*stage - self.lines+1, -(line+1 - self.lines/2+1), 0]
                    self.layout[self.node_id(line+1, stage, 1)] = [2*stage+1 - self.lines+1, -(line+1 - self.lines/2+1), 0]

        # Input and output nodes
        for line in range(0, self.lines):
            self.OddEven.add_node(self.input_node(line), weight = {'value': self.null_label})
            self.OddEven.add_edge(self.input_node(line), self.node_id(line, 0, 0), weight={"carry": self.null_label, "epoch": 0})
            self.OddEven.add_node(self.output_node(line), weight = {'value': self.null_label})
            if line == 0 or line == self.lines-1:
                self.OddEven.add_edge(self.node_id(line, self.lines-2, 1), self.output_node(line), weight={"carry": self.null_label, "epoch": 0})
            else:
                self.OddEven.add_edge(self.node_id(line, self.lines-1, 1), self.output_node(line), weight={"carry": self.null_label, "epoch": 0})
            self.layout[self.input_node(line)] = [0 - self.lines - 1+1, -(line - self.lines/2+1), 0]
            self.layout[self.output_node(line)] = [2*self.lines - self.lines - 2+3, -(line - self.lines/2+1), 0]

        # Inter-comparator connections
        for gap in range(0, self.lines-1):
            for line in range(1, self.lines-1):
                assert(self.node_id(line, gap, 1) in self.OddEven.nodes)
                assert(self.node_id(line, gap+1, 0) in self.OddEven.nodes)
                self.OddEven.add_edge(self.node_id(line, gap, 1), self.node_id(line, gap+1, 0), weight={"carry": self.null_label, "epoch": 0})
        for gap in range(0, self.lines-2, 2):
            assert(self.node_id(0, gap, 1) in self.OddEven.nodes)
            assert(self.node_id(0, gap+2, 0) in self.OddEven.nodes)
            self.OddEven.add_edge(self.node_id(0, gap, 1), self.node_id(0, gap+2, 0), weight={"carry": self.null_label, "epoch": 0})
            self.OddEven.add_edge(self.node_id(self.lines-1, gap, 1), self.node_id(self.lines-1, gap+2, 0), weight={"carry": self.null_label, "epoch": 0})

    def additional_items_to_draw(self):
        list = []
        list.append(('RectangleAroundNodes', [x for x in self.inputs()]))
        list.append(('RectangleAroundNodes', [x for x in self.outputs()]))
        return list

    def set_input(self, seq):
        self.seq = seq

    def get(self):
        return self.OddEven, self.layout

    def init_computation(self):
        for node, value in zip(self.inputs(), self.seq):
            self.OddEven.nodes[node]['weight']['value'] = str(value)
        self.epoch = 0

    def compute_iteration(self):
        some_updates = False
        for node in self.OddEven:
            if node in self.inputs():
                if self.epoch == 0:
                    for u, v, d in self.OddEven.out_edges(node, data=True):
                        assert(u==node)
                        if self.OddEven.nodes[node]['weight']['value'] != self.null_label:
                            some_updates = True
                            d['weight']['carry'] = self.OddEven.nodes[node]['weight']['value']
                            d['weight']['epoch'] = self.epoch+1
                else:
                    None
            else:
                if node in self.outputs():
                    in_a = None
                    for u, v, e in self.OddEven.in_edges(node, data=True):
                        in_a = e
                    assert(in_a != None)

                    if in_a['weight']['carry'] != self.null_label and in_a['weight']['epoch'] <= self.epoch:
                        some_updates = True
                        self.OddEven.nodes[node]['weight']['value'] = in_a['weight']['carry']
                        in_a['weight']['carry'] = self.null_label

                else:
                    line, stage, comp = self.items_from_node_id(node)
                    #print(f'node {node} ({line}, {stage}, {comp})')
                    if comp == 0: # First stage of the comparator
                        in_a = None
                        for u, v, e in self.OddEven.in_edges(node, data=True):
                            in_a = e

                        out = []
                        for u, v, e in self.OddEven.out_edges(node, data=True):
                            out.append(e)

                        assert(len(out) == 2)
                        assert(in_a != None)
                        if in_a['weight']['carry'] != self.null_label and in_a['weight']['epoch'] <= self.epoch:
                            some_updates = True
                            self.OddEven.nodes[node]['weight']['value'] = in_a['weight']['carry']
                            out[0]['weight']['carry'] = self.OddEven.nodes[node]['weight']['value']
                            out[0]['weight']['epoch'] = self.epoch+1
                            out[1]['weight']['carry'] = self.OddEven.nodes[node]['weight']['value']
                            out[1]['weight']['epoch'] = self.epoch+1
                            in_a['weight']['carry'] = self.null_label
                    else:
                        assert(comp==1)
                        out_a = None
                        for u, v, e in self.OddEven.out_edges(node, data=True):
                            out_a = e
                        ins = []
                        for u, v, e in self.OddEven.in_edges(node, data=True):
                            ins.append(e)
                        assert(len(ins) == 2)
                        assert(out_a != None)
                        if ins[0]['weight']['carry'] != self.null_label and ins[1]['weight']['carry'] != self.null_label and ins[0]['weight']['epoch'] <= self.epoch and ins[1]['weight']['epoch'] <= self.epoch:
                            #if stage%2==0:
                                if line%2==stage%2:
                                    # Up
                                    some_updates = True
                                    out_a['weight']['carry'] = str(min(int(ins[0]['weight']['carry']), int(ins[1]['weight']['carry'])))
                                    self.OddEven.nodes[node]['weight']['value'] = out_a['weight']['carry']
                                    out_a['weight']['epoch'] = self.epoch+1
                                    ins[0]['weight']['carry'] = self.null_label
                                    ins[1]['weight']['carry'] = self.null_label
                                else:
                                    # Down
                                    some_updates = True
                                    out_a['weight']['carry'] = str(max(int(ins[0]['weight']['carry']), int(ins[1]['weight']['carry'])))
                                    self.OddEven.nodes[node]['weight']['value'] = out_a['weight']['carry']
                                    out_a['weight']['epoch'] = self.epoch+1
                                    ins[0]['weight']['carry'] = self.null_label
                                    ins[1]['weight']['carry'] = self.null_label

        self.epoch = self.epoch + 1
        return some_updates


In [202]:
for l in range(1, 6):
    lines = 1<<l
    net = odd_even(lines)
    for line in range(0, lines, 2):
        for stage in range(0, lines, 2):
            for comp in range(0, 4):
                id = net.node_id(line, stage, comp)
                (l, s, c) = net.items_from_node_id(id)
                assert(line == l)
                assert(stage == s)
                assert(comp == c)
                id = net.node_id(line+1, stage, comp)
                (l, s, c) = net.items_from_node_id(id)
                assert(line+1 == l)
                assert(stage == s)
                assert(comp == c)
                id = net.node_id(line, stage+1, comp)
                (l, s, c) = net.items_from_node_id(id)
                assert(line == l)
                assert(stage+1 == s)
                assert(comp == c)
                id = net.node_id(line+1, stage+1, comp)
                (l, s, c) = net.items_from_node_id(id)
                assert(line+1 == l)
                assert(stage+1 == s)
                assert(comp == c)

print("All is OK!")


All is OK!


In [246]:
%%manim -qm ShowOddEven

config.frame_width = 25


class ShowOddEven(Scene):
    
    def construct(self):
        nbits = 8
        network = odd_even(nbits, delay=1)
        network.set_input([8, 7, 6, 5, 4, 3, 2 ,1])
        network.init_computation()
        generic_run(self, network)
        #print(network.read_output())

Manim Community v0.18.1

Iteration 0
Iteration 1


Iteration 2


Iteration 3


Iteration 4


...

Iteration 14


Iteration 15


Iteration 16


Iteration 17


Iteration 18


# Butterly Network (for DFT)

In [8]:
class butterly:
    def __init__(self, nlines, **kwargs):
        #print(kwargs)
        self.null_label = kwargs['null_label'] if 'null_label' in kwargs else '-'
        self.verbose = kwargs['verbose'] if 'verbose' in kwargs else False

        # nbits is the number of bits of the operation
        # N is the number of nodes in the tree (2^n-1). Nodes in the tree will be from 0 to 2^n-2
        self.lines = nlines
        self.N = 2*nlines-1

        self.Butterfly = nx.DiGraph()

        node = 0
        self.layout = {}
        for stage in range(0, math.log2(self.lines)):
            for line in range(0, self.lines):
                node1 = (line, stage, 0)
                node2 = (line, stage, 1)
                self.Butterfly.add_node(node1)
                self.Butterfly.add_node(node2)
            for line in range(0, self.lines):
                almost_node1 = line
                almost_node2 = flip_bit(node1, stage)
                if (almost_node2 > line):
                    self.Butterfly.add_edge((almost_node1, stage, 0), (flip_bit(almost_node1, stage), stage, 1))
                    self.layout[node] = [stage*0, line - self.lines/2, 0]
                else:
                    self.Butterfly.add_edge((almost_node1, stage, 1), (flip_bit(almost_node1, stage), stage, 0))
                    self.layout[node] = [stage*1+1, line - self.lines/2, 0]


    def get(self):
        return (self.Butterfly, self.layout)#, self.edges_labels_pos)

    def set_input(self, in_seq):
        self.in_seq = in_seq

    def init_computation(self):
        None

    def compute_iteration(self):
        None
    def read_output(self):
        None

In [10]:
nleaves = 16
test_seq = [x for x in range(0, nleaves)]
for i in range(0, 100):
	network = butterfly(nleaves, verbose = False)
	network.set_input(test_seq)
	network.init_computation()
	while network.compute_iteration():
		None
	res = network.read_output()
	assert(res[0] == test_seq[0])
	test_out = res[0]
	for x in range(1, nleaves):
		test_out = test_out + test_seq[x]
		#print(f'{test_out} == {test_seq[x]}?')
		assert(test_out == res[x])
	


NameError: name 'butterfly' is not defined

In [9]:
%%manim -qm ShowLinear

config.frame_width = 20
import random

class ShowLinear(Scene):
    
    def construct(self):
        nleaves = 8
        network = prefix_tree(nleaves, delay=1)
        network.set_input([random.randint(0, nleaves) for x in range(0, nleaves)])
        generic_run(self, network)
        #print(network.read_output())

Manim Community v0.18.1

NameError: name 'prefix_tree' is not defined

In [136]:
for i in range(-1, -9, -1):
    print(f'{i}')

-1
-2
-3
-4
-5
-6
-7
-8
